In [ ]:
import pandas as pd 

In [ ]:
dir = "C:/Users/UQAC/Downloads/hh101/hh101"

df =pd.read_csv(dir + "/" + "hh101.ann.features.csv")

df.info()

In [ ]:
#make parser for dataset <Date> <Time> <Sensor> <Translate01> <Translate02> <Message> <SensorType> <
def parse(line):
    line = line.split("\t")
    date = line[0]
    sensor = line[1]
    translate01 = line[2]
    translate02 = line[3]
    message = line[4]
    sensorType = line[5]
    return date, sensor, translate01, translate02, message, sensorType
    


In [106]:
#read h101.rawdata.txt line by line
file =dir + "/" + "hh101.rawdata.txt"

data= pd.DataFrame(columns=["Date", "Sensor", "Translate01", "Translate02", "Message", "SensorType"])

with open(file) as f:
    for line in f.readlines():
        #print(line)
        date, sensor, translate01, translate02, message, sensorType = parse(line)
        data.loc[len(data)] = [date, sensor, translate01, translate02, message, sensorType]


#convert date to datetime




KeyboardInterrupt: 

In [119]:
data["Date"] = pd.to_datetime(data["Date"])

In [117]:
def fen_gen(data,win_size,step_size):
    win_start = 0
    for i in range(0,len(data),step_size):
        win_end = win_start + win_size
        if win_end > len(data):
            break
        else:
            yield data[win_start:win_end]
            win_start += step_size 



    

In [123]:
MotionSensorlist = data[data["SensorType"]=="Control4-Motion\n"]["Sensor"].value_counts().index.tolist()
MotionSensorlist += data[data["SensorType"]=="Control4-MotionArea\n"]["Sensor"].value_counts().index.tolist()
LightSensorlist = data[data["SensorType"]=="Control4-LightSensor\n"]["Sensor"].value_counts().index.tolist()
DoorSensorlist = data[data["SensorType"]=="Control4-Door\n"]["Sensor"].value_counts().index.tolist()
TempSensorlist = data[data["SensorType"]=="Control4-Door\n"]["Sensor"].value_counts().index.tolist()

def compute_features(fen,last_features) :
    
    #heure 
    last_date = fen["Date"].iloc[-1]
    last_hour = last_date.hour + last_date.minute/60
    duration = (fen["Date"].iloc[-1] - fen["Date"].iloc[0]).seconds

    #jour de la semaine
    week_day = last_date.weekday()
    features = [last_date,last_hour, week_day, duration]
    #motion sensor
    
    i=len(features)
    for m in MotionSensorlist:
        if fen[fen["Sensor"]==m].shape[0] > 0:
            if fen[fen["Sensor"]==m].iloc[-1]["Message"] == "ON":
                features.append(1)
            else:
                features.append(-1)
        else:
            features.append(last_features[i])
        i+=1

    

    for d in DoorSensorlist:
        if fen[fen["Sensor"]==m].shape[0] > 0:
            if fen[fen["Sensor"]==m].iloc[-1]["Message"] == "OPEN":
                features.append(1)
            else:
                features.append(-1)
        else:
            features.append(last_features[i])
        i+=1
    
    
    for l in LightSensorlist:
        if fen[fen["Sensor"]==l].shape[0] > 0:
            features.append(int(fen[fen["Sensor"]==l].iloc[-1]["Message"])/100.)
        else:
            features.append(last_features[i])
        i+=1
    
    for t in TempSensorlist:
        if fen[fen["Sensor"]==l].shape[0] > 0:
            features.append((fen[fen["Sensor"]==l].iloc[-1]["Message"])/100.)
        else:
            features.append(last_features[i])
        i+=1
    
    
    
    
    return features



In [126]:
gen = fen_gen(data,30,15)

compute_features(next(gen),[0 for i in range(100)])

42

In [98]:
#print Sensors for sensorType= Contrl4-motion
data["Sensor"].value_counts().index

Index(['M008', 'MA015', 'MA014', 'M012', 'M003', 'LS008', 'MA016', 'MA013',
       'M009', 'LS015', 'LS014', 'LS012', 'M001', 'LS016', 'LS013', 'LS009',
       'M002', 'LS001', 'M010', 'LS003', 'LS005', 'LS011', 'M005', 'LS010',
       'M011', 'LS002', 'LS004', 'M004', 'M006', 'LS006', 'T101', 'T105',
       'D002', 'T104', 'T103', 'M007', 'LS007', 'BATP001', 'T102', 'D001',
       'BATP005', 'BATP008', 'BATP014', 'BATP002', 'BATP102', 'BATP015',
       'BATP003', 'BATP103', 'BATP012', 'BATP013', 'BATP010', 'BATP007',
       'BATP004', 'BATP011', 'BATP016', 'BATP009', 'BATP006', 'D003'],
      dtype='object')

In [109]:
data[data["SensorType"]=="Control4-Door\n"]["Sensor"].value_counts().index.tolist()


['D002', 'D001', 'D003']

In [ ]:
# faire un modele de detection d'anomalie
model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)


